In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['JAVA_HOME'] = '/Users/sb0487/Desktop/sb0487/pyspark/zulu15.32.15-ca-jdk15.0.3-macosx_aarch64/zulu-15.jdk/Contents/Home' 


In [22]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("ggplot")
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import cufflinks as cf
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import pandas as pd
import numpy as np
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from ipywidgets import interact, interact_manual
from pyspark.sql.window import Window
import pendulum as plm
from pyspark.sql.types import IntegerType
cf.go_offline(connected=True)
init_notebook_mode(connected=True)

In [23]:
from pyspark.sql import SparkSession as Session
from pyspark import SparkConf as Conf
from pyspark.sql import functions as F, Window as W
C = F.col

In [24]:
conf = (Conf()
        .set("spark.sql.sources.partitionOverwriteMode", "dynamic")
        .set("spark.driver.memory", "3g")
        .set("spark.sql.broadcastTimeout", "10200")
        .set("spark.driver.maxResultSize", "1g")
#         .set("spark.local.dir","/buffer") 指定shuffle write的路徑
       ) 

In [25]:
spark = (Session
         .builder
         .appName("tradddddd")
         .master("local[4]")# 核心數量
         .config(conf=conf)
         .getOrCreate())

In [26]:
# can see UI
spark

# feature extention

## test

In [71]:
sdf = spark.read.csv("/Users/sb0487/Desktop/sb0487/finance/project_stock/data/TXF_investor_chip_no_adjust.csv",header = True)


In [88]:
from pyspark.sql import DataFrame
def format_columns_name(sdf:DataFrame) -> DataFrame:
    format_columns_name_sdf = (
        sdf
        .withColumnRenamed("時間","Date")
        .withColumnRenamed("開盤價","Open")
        .withColumnRenamed("最高價","High")
        .withColumnRenamed("最低價","Low")
        .withColumnRenamed("收盤價","Close")
    )
    return format_columns_name_sdf

In [89]:
formatted_columns_name_sdf = format_columns_name(sdf)

In [92]:
def format_char_sdf(sdf:DataFrame, replace_char = "口") -> DataFrame:
    """將字元拿掉，並轉為IntegerType"""
    for column in sdf.columns[1:]:
        formatted_char_sdf = (
            sdf
            .withColumn(column,F.regexp_replace(C(column),replace_char, '').cast(IntegerType()))
        )
    return formatted_char_sdf

In [96]:
formatted_char_sdf = format_sdf(formatted_columns_name_sdf,"口")

In [97]:
formatted_char_sdf.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: integer (nullable = true)
 |-- High: integer (nullable = true)
 |-- Low: integer (nullable = true)
 |-- Close: integer (nullable = true)
 |-- 外資及陸資交易買口: integer (nullable = true)
 |-- 外資及陸資交易賣口: integer (nullable = true)
 |-- 外資及陸資交易淨口: integer (nullable = true)
 |-- 投信交易買口: integer (nullable = true)
 |-- 投信交易賣口: integer (nullable = true)
 |-- 投信交易淨口: integer (nullable = true)
 |-- 自營商交易買口: integer (nullable = true)
 |-- 自營商交易賣口: integer (nullable = true)
 |-- 自營商交易淨口: integer (nullable = true)
 |-- 外資及陸資未平倉買口: integer (nullable = true)
 |-- 外資及陸資未平倉賣口: integer (nullable = true)
 |-- 外資及陸資未平倉淨口: integer (nullable = true)
 |-- 投信未平倉買口: integer (nullable = true)
 |-- 投信未平倉賣口: integer (nullable = true)
 |-- 投信未平倉淨口: integer (nullable = true)
 |-- 自營商未平倉買口: integer (nullable = true)
 |-- 自營商未平倉賣口: integer (nullable = true)
 |-- 自營商未平倉淨口: integer (nullable = true)
 |-- 前十大法人買口: integer (nullable = true)
 |-- 前十大法人賣口: integer (nullable = tr

In [95]:
formatted_char_sdf.toPandas()

,Date,Open,High,Low,Close,外資及陸資交易買口,外資及陸資交易賣口,外資及陸資交易淨口,投信交易買口,投信交易賣口,...,自營商未平倉賣口,自營商未平倉淨口,前十大法人買口,前十大法人賣口,前十大特定法人買口,前十大特定法人賣口,前五大法人買口,前五大法人賣口,前五大特定法人買口,前五大特定法人賣口
0,2010-01-04,8203,8211,8101,8167,7558,7712,-154,622,608,...,4562,8635,28076,17001,17332,14601,17825,10913,8528,10913
1,2010-01-05,8212,8260,8125,8178,6648,6982,-334,757,821,...,3683,10041,30392,17852,19202,16641,18858,11270,9098,11270
2,2010-01-06,8197,8336,8179,8321,9592,7982,1610,479,353,...,5064,11905,33355,18153,21784,15299,21204,11417,9633,9865
3,2010-01-07,8335,8355,8230,8260,9757,17049,-7292,391,298,...,4366,13422,32146,20027,17796,18260,21378,12305,8482,12305
4,2010-01-08,8287,8297,8183,8277,5954,7364,-1410,495,562,...,5638,13363,32779,21796,17778,19849,22100,13084,8472,11137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,2021-06-21,17048,17149,16891,16963,81392,81266,126,37,965,...,4341,21080,50799,59786,45529,59786,37235,41919,37235,41919
2819,2021-06-22,17155,17180,16988,17026,57672,57277,395,31,95,...,5973,19442,51398,58973,45407,55948,37255,41580,37255,41580
2820,2021-06-23,17100,17319,17059,17267,67540,68947,-1407,1076,8,...,7005,18513,50034,60032,44638,60032,37366,42790,37366,42790
2821,2021-06-24,17298,17398,17261,17338,47153,46180,973,85,0,...,7474,17619,48941,59974,44085,56984,36683,42545,36683,42545


In [137]:
def extended_feature(sdf:DataFrame,columns = sdf.columns[5:],lag1 = True) -> DataFrame:
    """輸入想要擴展特徵的column與擴展的特徵，可能未來可加上sma值等"""
    if lag1:
        for column in columns:
            window = Window.partitionBy().orderBy("Date")
            sdf = (
                sdf
                .withColumn("prev_value",F.lag(column).over(window))
                .withColumn(f"∆{column}", F.when(F.isnull(C(column)-C("prev_value")), 0)
                                  .otherwise(C(column)-C("prev_value")))
              .drop("prev_value")
            )
        
    return sdf

In [138]:
extended_feature_sdf = extended_feature(formatted_char_sdf)

In [136]:
extended_feature_sdf.toPandas()

,Date,Open,High,Low,Close,外資及陸資交易買口,外資及陸資交易賣口,外資及陸資交易淨口,投信交易買口,投信交易賣口,...,∆自營商未平倉賣口,∆自營商未平倉淨口,∆前十大法人買口,∆前十大法人賣口,∆前十大特定法人買口,∆前十大特定法人賣口,∆前五大法人買口,∆前五大法人賣口,∆前五大特定法人買口,∆前五大特定法人賣口
0,2010-01-04,8203,8211,8101,8167,7558,7712,-154,622,608,...,0,0,0,0,0,0,0,0,0,0
1,2010-01-05,8212,8260,8125,8178,6648,6982,-334,757,821,...,-879,1406,2316,851,1870,2040,1033,357,570,357
2,2010-01-06,8197,8336,8179,8321,9592,7982,1610,479,353,...,1381,1864,2963,301,2582,-1342,2346,147,535,-1405
3,2010-01-07,8335,8355,8230,8260,9757,17049,-7292,391,298,...,-698,1517,-1209,1874,-3988,2961,174,888,-1151,2440
4,2010-01-08,8287,8297,8183,8277,5954,7364,-1410,495,562,...,1272,-59,633,1769,-18,1589,722,779,-10,-1168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,2021-06-21,17048,17149,16891,16963,81392,81266,126,37,965,...,889,110,1612,1161,936,1161,71,484,71,484
2819,2021-06-22,17155,17180,16988,17026,57672,57277,395,31,95,...,1632,-1638,599,-813,-122,-3838,20,-339,20,-339
2820,2021-06-23,17100,17319,17059,17267,67540,68947,-1407,1076,8,...,1032,-929,-1364,1059,-769,4084,111,1210,111,1210
2821,2021-06-24,17298,17398,17261,17338,47153,46180,973,85,0,...,469,-894,-1093,-58,-553,-3048,-683,-245,-683,-245


## production

In [46]:
import sys
sys.path.append("/Users/sb0487/Desktop/sb0487/finance/project_stock/")

In [47]:
from func_utils.utils import *

In [48]:
dir(utils)

['C',
 'Conf',
 'DataFrame',
 'F',
 'IntegerType',
 'Session',
 'W',
 '__builtins__',
 '__cached__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'extend_feature',
 'format_columns_name',
 'format_value']

In [49]:
sdf = spark.read.csv("/Users/sb0487/Desktop/sb0487/finance/project_stock/data/TXF_investor_chip_no_adjust.csv",header = True)


In [50]:
formatted_columns_name_sdf = format_columns_name(sdf)

In [51]:
formatted_columns_name_sdf.limit(10).toPandas()

,Date,Open,High,Low,Close,外資及陸資交易買口,外資及陸資交易賣口,外資及陸資交易淨口,投信交易買口,投信交易賣口,...,自營商未平倉賣口,自營商未平倉淨口,前十大法人買口,前十大法人賣口,前十大特定法人買口,前十大特定法人賣口,前五大法人買口,前五大法人賣口,前五大特定法人買口,前五大特定法人賣口
0,2010-01-04,8203,8211,8101,8167,7558口,7712口,-154口,622口,608口,...,4562口,8635口,28076口,17001口,17332口,14601口,17825口,10913口,8528口,10913口
1,2010-01-05,8212,8260,8125,8178,6648口,6982口,-334口,757口,821口,...,3683口,10041口,30392口,17852口,19202口,16641口,18858口,11270口,9098口,11270口
2,2010-01-06,8197,8336,8179,8321,9592口,7982口,1610口,479口,353口,...,5064口,11905口,33355口,18153口,21784口,15299口,21204口,11417口,9633口,9865口
3,2010-01-07,8335,8355,8230,8260,9757口,17049口,-7292口,391口,298口,...,4366口,13422口,32146口,20027口,17796口,18260口,21378口,12305口,8482口,12305口
4,2010-01-08,8287,8297,8183,8277,5954口,7364口,-1410口,495口,562口,...,5638口,13363口,32779口,21796口,17778口,19849口,22100口,13084口,8472口,11137口
5,2010-01-11,8278,8353,8251,8322,7785口,5325口,2460口,677口,533口,...,5833口,13719口,33862口,22808口,19986口,20766口,22674口,13655口,8798口,13655口
6,2010-01-12,8318,8337,8246,8309,6568口,7542口,-974口,360口,236口,...,6142口,14046口,34669口,23371口,20035口,21407口,23582口,13875口,8948口,13875口
7,2010-01-13,8259,8275,8191,8215,8394口,11025口,-2631口,295口,497口,...,8082口,11917口,35430口,25389口,18702口,21507口,24457口,14669口,12182口,14669口
8,2010-01-14,8246,8328,8236,8299,8366口,8409口,-43口,538口,363口,...,7975口,14764口,38127口,26945口,20157口,22922口,26511口,16122口,10602口,16122口
9,2010-01-15,8310,8385,8307,8367,9754口,10318口,-564口,419口,714口,...,8240口,17123口,40081口,27415口,20262口,21524口,28425口,16933口,10766口,16933口


In [52]:
formatted_value_sdf = format_value(formatted_columns_name_sdf)

In [53]:
formatted_value_sdf.limit(10).toPandas()

,Date,Open,High,Low,Close,外資及陸資交易買口,外資及陸資交易賣口,外資及陸資交易淨口,投信交易買口,投信交易賣口,...,自營商未平倉賣口,自營商未平倉淨口,前十大法人買口,前十大法人賣口,前十大特定法人買口,前十大特定法人賣口,前五大法人買口,前五大法人賣口,前五大特定法人買口,前五大特定法人賣口
0,2010-01-04,8203,8211,8101,8167,7558,7712,-154,622,608,...,4562,8635,28076,17001,17332,14601,17825,10913,8528,10913
1,2010-01-05,8212,8260,8125,8178,6648,6982,-334,757,821,...,3683,10041,30392,17852,19202,16641,18858,11270,9098,11270
2,2010-01-06,8197,8336,8179,8321,9592,7982,1610,479,353,...,5064,11905,33355,18153,21784,15299,21204,11417,9633,9865
3,2010-01-07,8335,8355,8230,8260,9757,17049,-7292,391,298,...,4366,13422,32146,20027,17796,18260,21378,12305,8482,12305
4,2010-01-08,8287,8297,8183,8277,5954,7364,-1410,495,562,...,5638,13363,32779,21796,17778,19849,22100,13084,8472,11137
5,2010-01-11,8278,8353,8251,8322,7785,5325,2460,677,533,...,5833,13719,33862,22808,19986,20766,22674,13655,8798,13655
6,2010-01-12,8318,8337,8246,8309,6568,7542,-974,360,236,...,6142,14046,34669,23371,20035,21407,23582,13875,8948,13875
7,2010-01-13,8259,8275,8191,8215,8394,11025,-2631,295,497,...,8082,11917,35430,25389,18702,21507,24457,14669,12182,14669
8,2010-01-14,8246,8328,8236,8299,8366,8409,-43,538,363,...,7975,14764,38127,26945,20157,22922,26511,16122,10602,16122
9,2010-01-15,8310,8385,8307,8367,9754,10318,-564,419,714,...,8240,17123,40081,27415,20262,21524,28425,16933,10766,16933


In [108]:
extended_feature_sdf = extend_feature(formatted_value_sdf,columns=["外資及陸資交易買口","外資及陸資交易賣口"],lag1=True)

In [110]:
extended_feature_sdf.toPandas()

,Date,Open,High,Low,Close,外資及陸資交易買口,外資及陸資交易賣口,外資及陸資交易淨口,投信交易買口,投信交易賣口,...,前十大法人買口,前十大法人賣口,前十大特定法人買口,前十大特定法人賣口,前五大法人買口,前五大法人賣口,前五大特定法人買口,前五大特定法人賣口,∆外資及陸資交易買口,∆外資及陸資交易賣口
0,2010-01-05,8212,8260,8125,8178,6648,6982,-334,757,821,...,30392,17852,19202,16641,18858,11270,9098,11270,-910,-730
1,2010-01-06,8197,8336,8179,8321,9592,7982,1610,479,353,...,33355,18153,21784,15299,21204,11417,9633,9865,2944,1000
2,2010-01-07,8335,8355,8230,8260,9757,17049,-7292,391,298,...,32146,20027,17796,18260,21378,12305,8482,12305,165,9067
3,2010-01-08,8287,8297,8183,8277,5954,7364,-1410,495,562,...,32779,21796,17778,19849,22100,13084,8472,11137,-3803,-9685
4,2010-01-11,8278,8353,8251,8322,7785,5325,2460,677,533,...,33862,22808,19986,20766,22674,13655,8798,13655,1831,-2039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2817,2021-06-21,17048,17149,16891,16963,81392,81266,126,37,965,...,50799,59786,45529,59786,37235,41919,37235,41919,27873,29224
2818,2021-06-22,17155,17180,16988,17026,57672,57277,395,31,95,...,51398,58973,45407,55948,37255,41580,37255,41580,-23720,-23989
2819,2021-06-23,17100,17319,17059,17267,67540,68947,-1407,1076,8,...,50034,60032,44638,60032,37366,42790,37366,42790,9868,11670
2820,2021-06-24,17298,17398,17261,17338,47153,46180,973,85,0,...,48941,59974,44085,56984,36683,42545,36683,42545,-20387,-22767
